In [42]:
import sys
sys.path.append('..')
import pandas as pd
from helper import load_args
import torch
from main_images import KERC2020
import numpy as np
from tqdm.notebook import tqdm

In [3]:
args = load_args(is_notebook=True, verbose=False)

args.train_csv = '/home/congvm/Dataset/dataset/train_faces.csv'
args.val_csv = '/home/congvm/Dataset/dataset/valid_faces.csv'
args.data_dir = '/home/congvm/Dataset/'

kerc2020 = KERC2020()
args, dataloader, model = kerc2020.experiment_setup(args)
model(torch.ones([3, 3, 224, 224]))

(tensor([[-0.0216],
         [ 0.1478],
         [-0.0003]], grad_fn=<AddmmBackward>),
 tensor([[-0.0200],
         [ 0.2527],
         [-0.0885]], grad_fn=<AddmmBackward>),
 tensor([[-0.0216],
         [ 0.0916],
         [ 0.1381]], grad_fn=<AddmmBackward>))

In [4]:
state_dict = torch.load('../logs/resnet50/version_0/checkpoints/epoch=9.ckpt')
model.load_state_dict(state_dict['state_dict'])
model = model.to('cuda')
model = model.eval()

In [51]:
df = pd.DataFrame(columns=['ta', 'tv', 'ts', 'pa', 'pv', 'ps'])

In [58]:
for batch_idx, batch in tqdm(enumerate(dataloader.val_dataloader()), total=len(dataloader.val_dataloader())):
    x = batch['image']
    arousal = batch['arousal']
    stress = batch['stress']
    valence = batch['valence']
    
    x = x.to('cuda')
    pred_arousal, pred_stress, pred_valence  = model(x)
    pred_arousal, pred_stress, pred_valence = pred_arousal.cpu().detach().numpy(), pred_stress.cpu().detach().numpy(), pred_valence.cpu().detach().numpy()

    values = np.stack([arousal, valence, stress, pred_arousal, pred_valence, pred_stress]).squeeze().T
    for i, value in enumerate(values):    
#         print(batch_idx*len(values) + i)
        df.loc[batch_idx*len(values) + i] = value

In [59]:
val_loader = dataloader.val_dataloader()

In [60]:
val_loader.dataset.data_df

,video_name,frame_files,face_files,valence,arousal,stress
0,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_0.jpg,dataset/faces/0637.mp4/face_0.jpg,4.3,5.4,5.6
1,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_1.jpg,dataset/faces/0637.mp4/face_1.jpg,4.3,5.4,5.6
2,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_2.jpg,dataset/faces/0637.mp4/face_2.jpg,4.3,5.4,5.6
3,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_3.jpg,dataset/faces/0637.mp4/face_3.jpg,4.3,5.4,5.6
4,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_4.jpg,dataset/faces/0637.mp4/face_4.jpg,4.3,5.4,5.6
...,...,...,...,...,...,...
20317,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_44.jpg,dataset/faces/0936.mp4/face_44.jpg,4.7,5.3,5.2
20318,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_45.jpg,dataset/faces/0936.mp4/face_45.jpg,4.7,5.3,5.2
20319,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_46.jpg,dataset/faces/0936.mp4/face_46.jpg,4.7,5.3,5.2
20320,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_47.jpg,dataset/faces/0936.mp4/face_47.jpg,4.7,5.3,5.2


In [53]:
df.to_csv('submission.csv', index=False)

In [62]:
data_df = val_loader.dataset.data_df

In [65]:
len(data_df)

20322

In [67]:
len(df)

20320

In [64]:
data_df.join(df*10)

,video_name,frame_files,face_files,valence,arousal,stress,ta,tv,ts,pa,pv,ps
0,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_0.jpg,dataset/faces/0637.mp4/face_0.jpg,4.3,5.4,5.6,5.4,4.3,5.6,2.746718,2.205911,2.756621
1,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_1.jpg,dataset/faces/0637.mp4/face_1.jpg,4.3,5.4,5.6,5.4,4.3,5.6,2.955133,2.253571,2.992276
2,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_2.jpg,dataset/faces/0637.mp4/face_2.jpg,4.3,5.4,5.6,5.4,4.3,5.6,2.889266,2.099309,2.879271
3,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_3.jpg,dataset/faces/0637.mp4/face_3.jpg,4.3,5.4,5.6,5.4,4.3,5.6,2.859286,1.770778,3.021554
4,./2020kerc/Val/0637.mp4,dataset/frames/0637.mp4/frame_4.jpg,dataset/faces/0637.mp4/face_4.jpg,4.3,5.4,5.6,5.4,4.3,5.6,2.872009,1.802747,2.969785
...,...,...,...,...,...,...,...,...,...,...,...,...
20317,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_44.jpg,dataset/faces/0936.mp4/face_44.jpg,4.7,5.3,5.2,5.3,4.7,5.2,2.815689,2.274427,2.994353
20318,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_45.jpg,dataset/faces/0936.mp4/face_45.jpg,4.7,5.3,5.2,5.3,4.7,5.2,2.757272,2.615216,2.905523
20319,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_46.jpg,dataset/faces/0936.mp4/face_46.jpg,4.7,5.3,5.2,5.3,4.7,5.2,2.959254,2.491701,3.085282
20320,./2020kerc/Val/0936.mp4,dataset/frames/0936.mp4/frame_47.jpg,dataset/faces/0936.mp4/face_47.jpg,4.7,5.3,5.2,NaN,NaN,NaN,NaN,NaN,NaN
